<a href="https://colab.research.google.com/github/splasherzz/food-allergen-detector/blob/main/Project%20Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AI for AI: Utilizing Artificial Intelligence for Allergen Identification in Food**

# **Project Summary**

This project aims to develop an AI model for allergen detection in food products using a multi-label classification approach. It will utilize separate Random Forest classifiers for each allergen class, treating them as independent binary classification tasks. This project addresses the limitations of traditional allergen detection methods, ensuring consumer safety and improving food allergen detection accuracy and food quality control processes. Using the "Food Ingredients and Allergens" dataset from Kaggle, the model will be trained to predict the presence of allergenic ingredients in different food products. The model will learn how to recognize patterns and associations between ingredients and allergens. The developed model will then be integrated into a web application for user-friendly allergen detection, where people with allergies can make informed decisions about the food they buy and consume. This project has the potential to benefit food manufacturers, the food service industry, regulatory bodies, and people with allergies, ultimately improving food safety and promoting informed food choices.

# **Data Collection**

We obtained the 2023 Kaggle dataset titled ["Food Ingredients & Allergens"](https://kaggle.com/datasets/uom190346a/food-ingredients-and-allergens) by Laksika Tharmalingam. We augmented data by manually adding 92 entries to the dataset, adding various food products and their allergen labels that we got from Google. The augmented dataset will then be used to train and test our AI model for food allergen detection.

## **Importing the Original Dataset**

The "Food Ingredients & Allergens" dataset is uploaded as a .csv file in our [GitHub repository](https://github.com/splasherzz/food-allergen-detector). This is imported and initialized in our notebook as `og_food`.

In [1951]:
import io
import pandas as pd

# initializing the dataset
data = 'https://raw.githubusercontent.com/splasherzz/food-allergen-detector/main/datasets/%5BOriginal%5D%20Food%20Ingredients%20and%20Allergens.csv'
og_food = pd.read_csv(data)
og_food.head()

,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens,Prediction
0,Almond Cookies,Almonds,Sugar,Butter,Flour,"Almonds, Wheat, Dairy",Contains
1,Almond Cookies,Almonds,Sugar,Butter,Flour,"Almonds, Wheat, Dairy",Contains
2,Chicken Noodle Soup,Chicken broth,None,None,Salt,"Chicken, Wheat, Celery",Contains
3,Chicken Noodle Soup,Chicken broth,None,None,Salt,"Chicken, Wheat, Celery",Contains
4,Cheddar Cheese,Cheese,None,None,Salt,Dairy,Contains


## **Dataset Features**

Below is a table of the dataset features and their respective descriptions.

<table>
  <tr>
    <th>Column Name<th/>
    <th>Description<th/>
  <tr/>

  <tr>
    <td>Food Product<td/>
    <td>Name of the food product<td/>
  <tr/>
  <tr>
    <td>Main Ingredient<td/>
    <td>Defining or distinctive ingredient of the food product<td/>
  <tr/>
  <tr>
    <td>Sweetener<td/>
    <td>Substance added to food or drink to impart the flavor of sweetness<td/>
  <tr/>
  <tr>
    <td>Fat/Oil<td/>
    <td>Lipids made from plants, animals, or synthetic compounds used when frying, baking, and preparing foods<td/>
  <tr/>
  <tr>
    <td>Seasoning<td/>
    <td>Salt, herbs, or spices added to food to enhance the flavour<td/>
  <tr/>
  <tr>
    <td>Allergens<td/>
    <td>Any normally harmless substance that causes an immediate allergic reaction in a susceptible person<td/>
  <tr/>
  <tr>
    <td>Prediction<td/>
    <td>Anticipated outcome of the model<td/>
  <tr/>
<table/>

# **Data Preprocessing**

Before augmenting the dataset, we cleaned the original dataset by doing type formatting and handling null values and duplicate entries.

## **Type Formatting**

The table provided below summarizes the possible data types that we may encounter when using Pandas.

<table>
  <tr>
    <th>Data type<th/>
    <th>Description<th/>
  <tr/>

  <tr>
    <td>object<td/>
    <td>Text or mixed numeric and non-numeric values<td/>
  <tr/>
  <tr>
    <td>int64<td/>
    <td>Integer numbers<td/>
  <tr/>
  <tr>
    <td>float64<td/>
    <td>Floating point numbers<td/>
  <tr/>
  <tr>
    <td>bool<td/>
    <td>True/False values<td/>
  <tr/>
  <tr>
    <td>datetime64<td/>
    <td>Date and time values<td/>
  <tr/>
  <tr>
    <td>timedelta[ns]<td/>
    <td>Differences between two datetimes<td/>
  <tr/>
  <tr>
    <td>category<td/>
    <td>Finite list of text values<td/>
  <tr/>
<table/>

Using `dtypes` on the dataset, we observed that all columns had the data type `object`, which is described as "text or mixed numeric and non-numeric values." We then perform type formatting to ensure all our columns have the type `category`, as it fits the description of a "finite list of text values." It is also appropriate for our dataset since our columns are different categories of the food product.

In [1952]:
# performing type formatting to change all columns data types into "category"
for item in og_food:
  if og_food[item].dtype == object:
    og_food[item] = og_food[item].astype('category')

og_food.dtypes

Food Product       category
Main Ingredient    category
Sweetener          category
Fat/Oil            category
Seasoning          category
Allergens          category
Prediction         category
dtype: object

## **Handling Duplicates & Null Values**

Before dropping duplicates, we first check if there are null values in the dataset. Only one column had a null value and it was in the `Prediction` column. 

In [1953]:
# handling null values
print("Total number of missing values in whole dataset:", og_food.isna().sum().sum())
print("\n")
print("Breakdown of which columns have missing values:\n", og_food.isna().sum())
print("\n")
print("Entry with null values:\n", og_food[og_food.isna().any(axis=1)])

Total number of missing values in whole dataset: 1


Breakdown of which columns have missing values:
 Food Product       0
Main Ingredient    0
Sweetener          0
Fat/Oil            0
Seasoning          0
Allergens          0
Prediction         1
dtype: int64


Entry with null values:
     Food Product Main Ingredient Sweetener Fat/Oil     Seasoning  \
338   Baked Ziti           Pasta      None  Cheese  Tomato sauce   

        Allergens Prediction  
338  Wheat, Dairy        NaN  


Upon manually checking the dataset for the row with null value (entry 338), it was a duplicate for the same entry with `Prediction` correctly filled up. Thus, we just drop this single row with null entry. We also drop the duplicates in the dataset, keeping only the first occurrence. 

In [1954]:
# dropping entry with null value
og_food.dropna(inplace=True)

# handling & dropping duplicates
og_food.drop_duplicates(keep='first', inplace=True)
og_food.head()

,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens,Prediction
0,Almond Cookies,Almonds,Sugar,Butter,Flour,"Almonds, Wheat, Dairy",Contains
2,Chicken Noodle Soup,Chicken broth,None,None,Salt,"Chicken, Wheat, Celery",Contains
4,Cheddar Cheese,Cheese,None,None,Salt,Dairy,Contains
5,Ranch Dressing,Buttermilk,Sugar,Vegetable oil,"Garlic, herbs",Dairy,Contains
6,Caramel Popcorn,Popcorn,Sugar,Butter,Salt,Dairy,Contains


After dropping the duplicates and a row with a null value, we check for the current shape of the dataset. As shown below, there are only 308 entries left out of the initial 400.

In [1955]:
og_food.shape

(308, 7)

## **Augmenting the Dataset**
We augment the dataset by downloading the cleaned/preprocessed .csv file and manually adding 92 more entries to reach the same number of rows as prior to the deletion. To procure the data, we looked for random food products, their ingredients, and their allergen labels on Google.
We first made sure that there were no duplicates by searching the food product in the dataset before adding it.

In [1956]:
# # saving it as a CSV file
# df = pd.DataFrame(og_food)
# df.to_csv("[Augmented] Food Ingredients and Allergens.csv", index=False) 

## **Importing the Augmented Dataset**

Then, we import the augmented dataset as `aug_food`, and verify if its shape is correct. The dataset is uploaded as a .csv file in our [GitHub repository](https://github.com/splasherzz/food-allergen-detector).

In [1957]:
data1 = 'https://raw.githubusercontent.com/splasherzz/food-allergen-detector/main/datasets/%5BAugmented%5D%20Food%20Ingredients%20and%20Allergens.csv'
aug_food = pd.read_csv(data1)

aug_food.head()

,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens,Prediction
0,Almond Cookies,Almonds,Sugar,Butter,Flour,"Almonds, Wheat, Dairy",Contains
1,Chicken Noodle Soup,Chicken broth,None,None,Salt,"Chicken, Wheat, Celery",Contains
2,Cheddar Cheese,Cheese,None,None,Salt,Dairy,Contains
3,Ranch Dressing,Buttermilk,Sugar,Vegetable oil,"Garlic, herbs",Dairy,Contains
4,Caramel Popcorn,Popcorn,Sugar,Butter,Salt,Dairy,Contains


## Lowercase Conversion
Lowercasing the whole dataframe is important for consistency and standardization. By converting all values to lowercase, we ensure that the data is consistent throughout, helping to avoid duplicate or mismatched entries

In [1958]:
aug_food = aug_food.applymap(lambda s: s.lower() if type(s) == str else s)
aug_food.head()

,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens,Prediction
0,almond cookies,almonds,sugar,butter,flour,"almonds, wheat, dairy",contains
1,chicken noodle soup,chicken broth,none,none,salt,"chicken, wheat, celery",contains
2,cheddar cheese,cheese,none,none,salt,dairy,contains
3,ranch dressing,buttermilk,sugar,vegetable oil,"garlic, herbs",dairy,contains
4,caramel popcorn,popcorn,sugar,butter,salt,dairy,contains


## **Preprocessed Dataset Features**

The data features in the preprocessed dataset represent different food products such as "Aloo Gobi" and "Apple Cider." These features are stored as unsigned 8-bit integers (`uint8`) for efficient memory usage as the binary values of the features can be accommodated within this range.

In [1960]:
aug_food.dtypes

Food Product       object
Main Ingredient    object
Sweetener          object
Fat/Oil            object
Seasoning          object
Allergens          object
Prediction         object
dtype: object

After augmenting, we check for the current shape of the dataset. As shown below, the number of entries are back to the initial 400 due to the additional 92 entries.

In [1961]:
aug_food.shape

(400, 7)

After cleaning and preprocessing the data, we can now proceed to modeling the data using multi-label classification and Random Forest Classifier.

## **Converting Categorical Features through Categorical Data Encoding**

After augmenting the dataset, we did categorical data encoding. To convert the qualitative values `'Does not contain'` and `'Contains'` into numerical representation, we respectively map the prediction values to 0 and 1 using `.map`. This is necessary for training our multilabel classification model which requires numerical input for making predictions.

In [1962]:
# mapping the prediction values to 0 or 1
aug_food['Prediction'] = aug_food['Prediction'].map({'contains': 1, 'does not contain': 0})

# showing that the prediction values are now set to 0/1
aug_food['Prediction']

0      1
1      1
2      1
3      1
4      1
      ..
395    1
396    1
397    1
398    1
399    1
Name: Prediction, Length: 400, dtype: int64

<!-- We then perform one-hot encoding on the augmented food dataset via `get_dummies`. This transforms categorical variables into binary (0/1) representations, allowing the classification model to understand and work with categorical data. Moreover, it enables the model to learn patterns and relationships between different food attributes which are crucial for accurate multilabel classification predictions. -->

In [ ]:
# # Performing one-hot encoding on categorical columns
# aug_food = pd.get_dummies(aug_food, drop_first=True)

# # Moving the 'Prediction' column to the last position
# aug_food.insert(len(aug_food), 'Prediction', aug_food.pop('Prediction'))

# # Showing the dataset
# aug_food.head()

We separately obtain the dummies for the `Seasoning` and `Allergens` features to allow for appropriate handling of the non-atomic values. By splitting the multivalued cells into separate values via `get_dummies()`, we create individual binary columns for each seasoning and allergen. This ensures that the resulting dummy variables correctly represent the presence or absence of each specific seasoning or allergen in the dataset, rather than incorrectly representing concatenated strings. 

We first obtain dummy variables for the `Allergens` column which results in binary columns representing the presence or absence of each unique allergen.

In [1963]:
aug_allerg = aug_food['Allergens'].str.replace(' ', '').str.get_dummies(',')
aug_allerg = aug_allerg.add_prefix('Allergens_')
aug_allerg.head()

,Allergens_alcohol,Allergens_almonds,Allergens_anchovies,Allergens_beef,Allergens_calamari,Allergens_celery,Allergens_chicken,Allergens_cocoa,Allergens_coconut,Allergens_crab,...,Allergens_oats,Allergens_peanuts,Allergens_pinenuts,Allergens_pork,Allergens_rice,Allergens_shellfish,Allergens_shrimp,Allergens_soybeans,Allergens_strawberries,Allergens_wheat
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Secondly, we obtain dummy variables separately for the `Seasoning` column which results in binary columns representing the presence or absence of each unique allergen.

In [1964]:
aug_season = aug_food['Seasoning'].str.replace(' ', '').str.get_dummies(',')
aug_season = aug_season.add_prefix('Seasoning_')
aug_season.head()

,Seasoning_alfredosauce,Seasoning_almondmilk,Seasoning_almonds,Seasoning_apple,Seasoning_applecider,Seasoning_applejuice,Seasoning_arboriorice,Seasoning_avocado,Seasoning_bacon,Seasoning_bakingpowder,...,Seasoning_vegetablebroth,Seasoning_vegetableoil,Seasoning_vegetables,Seasoning_vinaigrette,Seasoning_vinegar,Seasoning_walnuts,Seasoning_wheatflour,Seasoning_yeast,Seasoning_yogurt,Seasoning_zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The remaining columns of the food allergen dataframe, excluding `Allergens` and `Seasoning`, are transformed into dummy variables.

In [1965]:
aug_food.drop(['Seasoning','Allergens'], axis=1, inplace=True)
# Performing one-hot encoding on categorical columns
aug_food = pd.get_dummies(aug_food, drop_first=True)

# Showing the dataset
aug_food.head()

,Prediction,Food Product_aloo gobi,Food Product_aloo paratha,Food Product_apple,Food Product_apple cider,Food Product_apple crisp,Food Product_apple pie,Food Product_apple sauce,Food Product_apple tart,Food Product_arabic fattoush,...,Fat/Oil_peanut oil,Fat/Oil_pesto sauce,Fat/Oil_sesame oil,Fat/Oil_sour cream,Fat/Oil_soy sauce,Fat/Oil_vegetable oil,Fat/Oil_water,Fat/Oil_whipped cream,Fat/Oil_yogurt,"Fat/Oil_yogurt (milk, cultures)"
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Finally, we combine the three dataframes together into the desired single encoded dataframe using the `.join()` method. This consolidated dataframe can then be used for subsequent modeling.

In [1966]:
aug_food = aug_food.join(aug_season).join(aug_allerg)

# Moving the 'Prediction' column to the last position
aug_food.insert(len(aug_food.columns) - 1, 'Prediction', aug_food.pop('Prediction'))
aug_food.head()

,Food Product_aloo gobi,Food Product_aloo paratha,Food Product_apple,Food Product_apple cider,Food Product_apple crisp,Food Product_apple pie,Food Product_apple sauce,Food Product_apple tart,Food Product_arabic fattoush,Food Product_arancini,...,Allergens_peanuts,Allergens_pinenuts,Allergens_pork,Allergens_rice,Allergens_shellfish,Allergens_shrimp,Allergens_soybeans,Allergens_strawberries,Allergens_wheat,Prediction
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
397,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
398,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1


# **Data Modeling**

To begin our data modeling, we instantiate a **random forest classifier**, an ensemble-based machine learning algorithm commonly used for classification tasks. The classifier is configured as follows:
- **100** decision trees
- maximum depth of **10**
- random seed of **42**

In [1967]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

We assign the variable `X` to the `aug_food` dataframe with the `Prediction` column dropped, representing the input features. Similarly, the variable `y` is assigned to the `Prediction` column, representing the target variable.

In [1968]:
from sklearn.model_selection import train_test_split

X = aug_food.drop('Prediction', axis = 1)
y = aug_food['Prediction']

The next step is to split the data into training and testing subsets via the `train_test_split` function. The split is performed such that 70% of the data is assigned to the training set, while 30% is assigned to the testing set.

In [1969]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3)

In [1970]:
classifier.fit(X_train,y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [1971]:
classifier.score(X_test, y_test)

0.9916666666666667

In [1972]:
from sklearn.metrics import classification_report, confusion_matrix
pred = classifier.predict(X_test)
print('Confusion Matrix: \n', confusion_matrix(y_test,pred))

Confusion Matrix: 
 [[34  0]
 [ 1 85]]


In [1973]:
print('Classification Report: \n', classification_report(y_test,pred))

Classification Report: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.99        34
           1       1.00      0.99      0.99        86

    accuracy                           0.99       120
   macro avg       0.99      0.99      0.99       120
weighted avg       0.99      0.99      0.99       120



In [1974]:
from sklearn.metrics import accuracy_score
# Evaluate performance on the training set
train_predictions = classifier.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)
print("Training Accuracy:", train_accuracy)

# Evaluate performance on the validation set
test_predictions = classifier.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)
print("Testing Accuracy:", test_accuracy)

Training Accuracy: 0.9857142857142858
Testing Accuracy: 0.9916666666666667


## **Saving the Model**



In [1926]:
import pickle

filename = "model.pkl"

# save model
# pickle.dump(classifier, open(filename, "wb"))